In [18]:
import sys
# Append the directory to your python path using sys
sys.path.append("/content/drive/MyDrive/diepolder's paper/library_pipeline")

In [14]:

# Append the directory to your python path using sys
sys.path.append("/content/drive/MyDrive/diepolder's paper/library")

In [16]:
!pip install pymongo

In [ ]:
!ssh -N -L 27017:localhost:27017 med@137.194.217.238 &


ssh: connect to host 137.194.217.238 port 22: Connection timed out


In [20]:
import logging
import pandas as pd
# import ExtractDrtTrips as edt
# import PlansManipulation
import SpacialOperations as sops
import ExtractHubs
import Legs2df
import InterpolateTrips
import GTFSpreprocessing
import createGTFS
import importCells
import joinAndexportGTFS


ModuleNotFoundError: No module named 'SpacialOperations'

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
!pip install geojson #pyinterpolate #

In [23]:
# Constants
legs_input_path = "/content/drive/MyDrive/diepolder's paper/library_pipeline/Paris/output/output_legs.csv" # MATSim output legs csv file
studyarea_input_path = "/content/drive/MyDrive/diepolder's paper/library_pipeline/Paris/shp/cps.shp" # shapefile .shp for the DRT study area
popArea_input_path = "/content/drive/MyDrive/diepolder's paper/library_pipeline/Paris/shp/parisPopArea.shp"
gtfs_input_dir = "/content/drive/MyDrive/diepolder's paper/library_pipeline/Paris/gtfs" # path to where gtfs.zip is stored

# EPSG code !!ONLY NUMBER!! EPSG:3035 --> 3035
crs_import = 2154 # crs of the raw imported data
crs_working = 2154 # crs used for the processing (can be same as crs_import)

# Toggle for preprocessing steps
# GTFS preprocessing to get hexagonal grid
#      --> needed once, can be skipped if GTFScells.csv file exists
toggle_GTFSprep = True
GTFScells_path = "/content/drive/MyDrive/diepolder's paper/library_pipeline/Paris/output/GTFScells_drt.csv"

# Simulation & analysis start and end time
# Interpolation will only be done in this time window
start_time = '07:00:00'
end_time = '19:00:00'
# Length of each analysis interval
deltaT = 1 # in hours (h), default 1h

In [ ]:
# Setup of the logging module
logging.basicConfig(filename='Paris/output/Plans2GTFS.log',
                    level=logging.DEBUG)
console = logging.StreamHandler()
console.setLevel(logging.INFO)
logging.getLogger('').addHandler(console)

In [24]:
import Legs2df
df = Legs2df.run(legs_input_path)

ModuleNotFoundError: No module named 'Legs2df'

In [ ]:
df

,agentID,trip_id,<route>_distance,start_link,start_x,start_y,end_link,end_x,end_y,vehicle_id,<leg>_trav_time,wait_time,<leg>_dep_time
272,10004937,10004937_1,4798,537171,640676.987241,6.845386e+06,255857,638650.536115,6.848612e+06,vehicle_drt_93,443,277,28822
1215,1001879,1001879_1,2394,30347,645334.224928,6.847593e+06,249276,645178.745516,6.848194e+06,vehicle_drt_262,417,61,27593
3190,10035536,10035536_2,780,373905,634006.318125,6.844592e+06,418142,633676.048264,6.844983e+06,vehicle_drt_139,136,161,53076
3579,10038603,10038603_3,734,28293,642930.633995,6.849632e+06,523336,643382.681350,6.849181e+06,vehicle_drt_22,121,96,41275
4260,10043934,10043934_1,1141,52986,644475.791845,6.846566e+06,492973,644505.725852,6.845947e+06,vehicle_drt_259,208,16,28345
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4173637,998386,998386_1,10051,363799,642117.061658,6.845355e+06,246639,642372.295495,6.843694e+06,vehicle_drt_167,1557,198,30118
4174587,9998790,9998790_2,6887,255857,638650.536115,6.848612e+06,537171,640676.987241,6.845386e+06,vehicle_drt_125,682,178,56718
4174600,999887,999887_1,2818,13460,639578.460420,6.844363e+06,553637,641211.307923,6.842971e+06,vehicle_drt_39,483,551,30480
4174612,9998939,9998939_1,4798,537171,640676.987241,6.845386e+06,255857,638650.536115,6.848612e+06,vehicle_drt_101,384,457,31222


In [ ]:
edt.convertCRS(df,crs_import, crs_working,'start_x', 'start_y','end_x', 'end_y')

In [ ]:
edt.constructWktPoint(df, 'start_x', 'start_y', 'origin')
edt.constructWktPoint(df, 'end_x', 'end_y', 'destination')

In [ ]:
df = sops.createGeoDataFrame(df['origin'], 'EPSG:' + str(crs_working), df)

In [ ]:
df = df.to_crs(crs=crs_working)

In [ ]:
df

,agentID,trip_id,<route>_distance,start_link,start_x,start_y,end_link,end_x,end_y,vehicle_id,<leg>_trav_time,wait_time,<leg>_dep_time,origin,destination,geometry
272,10004937,10004937_1,4798,537171,640676.987241,6.845386e+06,255857,638650.536115,6.848612e+06,vehicle_drt_93,443,277,28822,POINT (640676.9872406526 6845385.660893883),POINT (638650.5361152911 6848611.9949821625),POINT (640676.987 6845385.661)
1215,1001879,1001879_1,2394,30347,645334.224928,6.847593e+06,249276,645178.745516,6.848194e+06,vehicle_drt_262,417,61,27593,POINT (645334.2249275153 6847592.524996355),POINT (645178.7455157 6848193.557723006),POINT (645334.225 6847592.525)
3190,10035536,10035536_2,780,373905,634006.318125,6.844592e+06,418142,633676.048264,6.844983e+06,vehicle_drt_139,136,161,53076,POINT (634006.3181254587 6844591.817313871),POINT (633676.0482636443 6844983.164553668),POINT (634006.318 6844591.817)
3579,10038603,10038603_3,734,28293,642930.633995,6.849632e+06,523336,643382.681350,6.849181e+06,vehicle_drt_22,121,96,41275,POINT (642930.6339947096 6849631.82615489),POINT (643382.6813501306 6849180.759473929),POINT (642930.634 6849631.826)
4260,10043934,10043934_1,1141,52986,644475.791845,6.846566e+06,492973,644505.725852,6.845947e+06,vehicle_drt_259,208,16,28345,POINT (644475.7918446767 6846565.807303847),POINT (644505.725852222 6845946.915065141),POINT (644475.792 6846565.807)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4173637,998386,998386_1,10051,363799,642117.061658,6.845355e+06,246639,642372.295495,6.843694e+06,vehicle_drt_167,1557,198,30118,POINT (642117.061658004 6845354.874643082),POINT (642372.2954946852 6843693.965287579),POINT (642117.062 6845354.875)
4174587,9998790,9998790_2,6887,255857,638650.536115,6.848612e+06,537171,640676.987241,6.845386e+06,vehicle_drt_125,682,178,56718,POINT (638650.5361152911 6848611.9949821625),POINT (640676.9872406526 6845385.660893883),POINT (638650.536 6848611.995)
4174600,999887,999887_1,2818,13460,639578.460420,6.844363e+06,553637,641211.307923,6.842971e+06,vehicle_drt_39,483,551,30480,POINT (639578.4604200339 6844363.259593032),POINT (641211.3079227756 6842971.122922318),POINT (639578.460 6844363.260)
4174612,9998939,9998939_1,4798,537171,640676.987241,6.845386e+06,255857,638650.536115,6.848612e+06,vehicle_drt_101,384,457,31222,POINT (640676.9872406526 6845385.660893883),POINT (638650.5361152911 6848611.9949821625),POINT (640676.987 6845385.661)


In [ ]:
DfHubs = ExtractHubs.run(df)

In [ ]:
DfHubs

,position,Count,name
0,POINT (633676.0482636443 6844983.164553668),198,Hub_0
1,POINT (636454.5534391978 6844596.322888183),730,Hub_1
2,POINT (637569.4661344511 6844167.124794338),185,Hub_2
3,POINT (638478.3893558099 6844294.495396683),421,Hub_3
4,POINT (639578.4604200339 6844363.259593032),829,Hub_4
5,POINT (640676.9872406526 6845385.660893883),427,Hub_5
6,POINT (642117.061658004 6845354.874643082),257,Hub_6
7,POINT (643382.6813501306 6849180.759473929),763,Hub_7
8,POINT (643881.6021442156 6845710.885985972),603,Hub_8
9,POINT (644475.7918446767 6846565.807303847),1243,Hub_9


In [ ]:
ExtractHubs.assignHub2Trip(DfHubs, df)

In [ ]:
DfHubs.to_csv("/content/drive/MyDrive/diepolder's paper/library_pipeline/Paris/output/drt_hubs.csv",sep=';')

In [ ]:
if toggle_GTFSprep: GTFSpreprocessing.run(GTFScells_path)

In [ ]:
CellsDF = importCells.run(GTFScells_path, crs_working)

In [ ]:
CellsDF

,geometry,Centroids,Centroid_X,Centroid_Y
0,"POLYGON ((626030.000 6830989.000, 625786.000 6...",POINT (625530.2660927754 6830995.862207755),625530.266093,6.830996e+06
1,"POLYGON ((626041.000 6831855.000, 625797.000 6...",POINT (625541.3340085049 6831861.534175934),625541.334009,6.831862e+06
2,"POLYGON ((626052.000 6832721.000, 625808.000 6...",POINT (625552.4019943832 6832727.211630778),625552.401994,6.832727e+06
3,"POLYGON ((626063.000 6833587.000, 625819.000 6...",POINT (625563.4700506227 6833592.894588898),625563.470051,6.833593e+06
4,"POLYGON ((626074.000 6834452.000, 625830.000 6...",POINT (625574.538177436 6834458.583066905),625574.538177,6.834459e+06
...,...,...,...,...
1789,"POLYGON ((654795.000 6866196.000, 654550.000 6...",POINT (654298.5312135745 6866200.329602678),654298.531214,6.866200e+06
1790,"POLYGON ((654802.000 6867062.000, 654557.000 6...",POINT (654305.3670352886 6867066.283966949),654305.367035,6.867066e+06
1791,"POLYGON ((654809.000 6867928.000, 654564.000 6...",POINT (654312.2029057125 6867932.24450173),654312.202906,6.867932e+06
1792,"POLYGON ((654816.000 6868794.000, 654571.000 6...",POINT (654319.0388249785 6868798.211223764),654319.038825,6.868798e+06


In [ ]:
# @title spatialoperations.py
# @ spatial_operation
import logging
import math
from shapely.geometry import Polygon
import shapefile as shp
import pyproj
import tqdm
import pandas as pd
import geopandas as gpd
import logging
import numpy as np

def runGridCreation(studyarea_input_path, gridSize):
    bBox = extractBoundingBoxFromShp(studyarea_input_path)
    polygons, centroids, Centroids_X , Centroids_Y = createSpatialGrid(bBox, gridSize)
    CellsDF = createGeoDataFrame(polygons, 'EPSG:3035', pd.DataFrame())
    CellsDF['Centroids'] = centroids
    CellsDF['Centroid_X'] = Centroids_X
    CellsDF['Centroid_Y'] = Centroids_Y
    CellsDF = removeExcessGrid(CellsDF, studyarea_input_path, 3035)
    return CellsDF

def extractBoundingBoxFromShp(shapeFilePath):
    proj = pyproj.Transformer.from_crs(4326, 3035, always_xy=True)
    shape = shp.Reader(shapeFilePath)
    raw = shape.bbox
    x1, y1 = proj.transform(raw[0], raw[1])
    x2, y2 = proj.transform(raw[2], raw[3])
    return [int(x1),int(y1),int(x2),int(y2)]

def extractBoundingBoxFromPointCloud(df):
    xMin = df['start_x'].min()
    xMax = df['start_x'].max()
    yMin = df['start_y'].min()
    yMax = df['start_y'].max()
    return[int(xMin),int(yMin), int(xMax), int(yMax)]

def getRadiusofPointCloud(df):
    bbox = extractBoundingBoxFromPointCloud(df)
    diagonal = math.sqrt(math.pow(bbox[2]-bbox[0],2) + math.pow(bbox[1]-bbox[3],2))
    radius = diagonal/2 * (2/3)
    return radius

def pointinCircle(radius,center,point):
    dist = math.sqrt(math.pow(center[0]-point[0],2) + math.pow(center[1]-point[1],2))
    return dist <= radius


def getHexagonCenterpoints(bbox, cellSize):
    xmin, ymin, xmax, ymax = bbox
    xmid = xmin + (xmax - xmin) / 2
    ymid = ymin + (ymax - ymin) / 2
    X = []
    xVals = [xmid]
    for xdif in range(0, int((xmax - xmin) / 2), cellSize):
        xVals.append(xVals[-1] + cellSize)
        xVals.insert(0, xVals[0] - cellSize)
    X.append(xVals)
    xVals = []
    for xdif in range(0, int((xmax - xmin) / 2), cellSize):
        if not xVals:  # check if list is empty
            xVals.append(xmid + cellSize / 2)
            xVals.insert(0, xmid - cellSize / 2)
        else:
            xVals.append(xVals[-1] + cellSize)
            xVals.insert(0, xVals[0] - cellSize)
    X.append(xVals)
    Y = [ymid]
    deltaY = int(round((1.5*cellSize)/math.sqrt(3),0))
    for ydif in range(0, int((ymax - ymin) / 2), deltaY):
        Y.append(Y[-1] + deltaY)
        Y.insert(0, Y[0] - deltaY)
    return X,Y

def constructHexagonWKT(x,y,r):
    R = (r*2)/(math.sqrt(3))
    X = [x+0,x+r,x+r,x+0,x-r,x-r]
    Y = [y+R,y+R/2,y-R/2,y-R,y-R/2,y+R/2]
    wkt = 'POLYGON (('
    for i in range(0,6,1):
        wkt = wkt + str(int(round(X[i],0))) + ' ' + str(int(round(Y[i],0))) + ', '
    wkt = wkt + str(int(round(X[0],0))) + ' ' + str(int(round(Y[0],0))) + '))'
    return wkt

def createSpatialGrid(bbox, cellSize):
    X, Y = getHexagonCenterpoints(bbox, cellSize)
    Polygons = []
    Centroids = []
    Centroids_X = []
    Centroids_Y = []
    progress = tqdm.tqdm(total=((len(Y)-1)/2), desc='Progress', position=0)
    for iY in range(0,len(Y)-1,2):
        progress.update(1)
        y1 = Y[iY]
        y2 = Y[iY+1]
        for iX in range(0,int(min(len(X[0]),len(X[1]))),1):
            x1 = X[0][iX]
            x2 = X[1][iX]
            r = cellSize/2
            h1 = constructHexagonWKT(x1, y1, r)
            h2 = constructHexagonWKT(x2, y2, r)
            p1 = 'POINT ('+str(x1)+' '+str(y1)+')'
            p2 = 'POINT (' + str(x2) + ' ' + str(y2) + ')'
            Centroids.append(p1)
            Centroids.append(p2)
            Polygons.append(h1)
            Polygons.append(h2)
            Centroids_X.append(x1)
            Centroids_X.append(x2)
            Centroids_Y.append(y1)
            Centroids_Y.append(y2)
    return Polygons, Centroids , Centroids_X, Centroids_Y

def createGeoDataFrame(geodata, crs, df):
    gs = gpd.GeoSeries.from_wkt(geodata,crs=crs)
    return gpd.GeoDataFrame(df, geometry=gs, crs=crs)

def removeExcessGrid(gdf, shp_studyArea, crs, shp_popArea, cutPopArea):
    logging.info('Removing hexagons outside of the regarded study area')
    df = gpd.read_file(shp_studyArea)
    df = df.to_crs(epsg=crs)
    studyArea = df['geometry'][0]
    polygon = Polygon(studyArea)
    gdf['inStudyArea'] = gdf['geometry'].overlaps(polygon) | gdf['geometry'].within(polygon)
    gdf = gdf[gdf['inStudyArea'] == True]
    #gdf.insert(loc=0,column='cell_id',value=range(0, len(gdf)))
    #gdf.set_index('cell_id')
    if cutPopArea:
        logging.info('Removing hexagons outside of populated area')
        df = gpd.read_file(shp_popArea)
        df = df.to_crs(epsg=crs)
        popArea = df['geometry'][0]
        polygon = Polygon(popArea)
        gdf['inStudyArea'] = gdf['geometry'].overlaps(polygon) | gdf['geometry'].within(polygon)
        gdf = gdf[gdf['inStudyArea'] == True]
    gdf.insert(loc=0, column='cell_id', value=range(0, len(gdf)))
    gdf.set_index('cell_id')

    return gdf

def countTripsPerCell(cellGdf,tripGdf):
    logging.info('Counting trips originating in each cell of hexagonal grid')
    workingDF = gpd.sjoin(tripGdf, cellGdf, how='inner', predicate='intersects')
    counts = workingDF.groupby(['cell_id'], as_index=False, )['agentID'].count()
    counts.columns = ['cell_id', 'origin_count']
    cellGdf = cellGdf.merge(counts, on='cell_id', how='left')
    cellGdf['origin_count'].fillna(0, inplace=True)
    return cellGdf

def addCellId2Trip(cellGdf,tripGdf):
    logging.info('Assigning corresponding cell to origin of trips')
    workingDF = gpd.sjoin(tripGdf, cellGdf, how='inner', predicate='intersects')
    tripGdf['originCell'] = workingDF['cell_id']

def aggregateTripsByCell(tripGdf,cellGdf,hubsDF):
    workingDF = tripGdf[['originCell','<leg>_trav_time','Hub']]
    workingDF = workingDF.groupby(['originCell','Hub'],as_index=False)['<leg>_trav_time'].mean()
    workingDF = pd.merge(cellGdf, workingDF, how='left', left_on='cell_id', right_on='originCell', validate='1:m')
    workingDF.sort_values(by='cell_id').reset_index(inplace=True)
    workingDF = pd.merge(workingDF,hubsDF[['position', 'name']],how='left',left_on='Hub',right_on='name')
    workingDF.drop(columns=['name', 'inStudyArea','originCell'], inplace=True)
    workingDF['position'] = workingDF['position'].fillna(0)
    return workingDF

def Points2Line(df,col1,col2):
    routes = []
    for i, row in df.iterrows():
        if row[col2] != 0:
            p1 = row[col1].strip('POINT (')
            p1 = p1.strip(')')
            p2 = row[col2].strip('POINT (').strip(')')
            p2 = p2.strip(')')
            r = 'LINESTRING (' + p1 + ', ' + p2 +')'
            routes.append(r)
        else:
            routes.append('')
    df['Routes'] = routes



In [ ]:
CellsDF = removeExcessGrid(CellsDF, studyarea_input_path, crs_working, popArea_input_path, False)
CellsDF

,cell_id,geometry,Centroids,Centroid_X,Centroid_Y,inStudyArea
476,0,"POLYGON ((633685.000 6844751.000, 633440.000 6...",POINT (633186.2403781626 6844756.47498971),633186.240378,6.844756e+06,True
519,1,"POLYGON ((634399.000 6841712.000, 634154.000 6...",POINT (633899.683422456 6841717.750894942),633899.683422,6.841718e+06,True
520,2,"POLYGON ((634408.000 6842578.000, 634164.000 6...",POINT (633909.5265291672 6842583.507444308),633909.526529,6.842584e+06,True
521,3,"POLYGON ((634418.000 6843444.000, 634174.000 6...",POINT (633919.3697006224 6843449.269688284),633919.369701,6.843449e+06,True
522,4,"POLYGON ((634428.000 6844309.000, 634183.000 6...",POINT (633929.2129370112 6844315.037643521),633929.212937,6.844315e+06,True
...,...,...,...,...,...,...
1630,351,"POLYGON ((652407.000 6847598.000, 652162.000 6...",POINT (651908.9867740034 6847601.907483266),651908.986774,6.847602e+06,True
1631,352,"POLYGON ((652414.000 6848464.000, 652169.000 6...",POINT (651916.1558914991 6848467.73052034),651916.155891,6.848468e+06,True
1632,353,"POLYGON ((652422.000 6849329.000, 652176.000 6...",POINT (651923.3250571139 6849333.5593688125),651923.325057,6.849334e+06,True
1675,354,"POLYGON ((653151.000 6847159.000, 652906.000 6...",POINT (652652.9857727668 6847162.8561709905),652652.985773,6.847163e+06,True


In [ ]:
CellsDF = countTripsPerCell(CellsDF, df)
CellsDF

,cell_id,geometry,Centroids,Centroid_X,Centroid_Y,inStudyArea,origin_count
0,0,"POLYGON ((633685.000 6844751.000, 633440.000 6...",POINT (633186.2403781626 6844756.47498971),633186.240378,6.844756e+06,True,2.0
1,1,"POLYGON ((634399.000 6841712.000, 634154.000 6...",POINT (633899.683422456 6841717.750894942),633899.683422,6.841718e+06,True,0.0
2,2,"POLYGON ((634408.000 6842578.000, 634164.000 6...",POINT (633909.5265291672 6842583.507444308),633909.526529,6.842584e+06,True,1.0
3,3,"POLYGON ((634418.000 6843444.000, 634174.000 6...",POINT (633919.3697006224 6843449.269688284),633919.369701,6.843449e+06,True,1.0
4,4,"POLYGON ((634428.000 6844309.000, 634183.000 6...",POINT (633929.2129370112 6844315.037643521),633929.212937,6.844315e+06,True,55.0
...,...,...,...,...,...,...,...
351,351,"POLYGON ((652407.000 6847598.000, 652162.000 6...",POINT (651908.9867740034 6847601.907483266),651908.986774,6.847602e+06,True,0.0
352,352,"POLYGON ((652414.000 6848464.000, 652169.000 6...",POINT (651916.1558914991 6848467.73052034),651916.155891,6.848468e+06,True,0.0
353,353,"POLYGON ((652422.000 6849329.000, 652176.000 6...",POINT (651923.3250571139 6849333.5593688125),651923.325057,6.849334e+06,True,0.0
354,354,"POLYGON ((653151.000 6847159.000, 652906.000 6...",POINT (652652.9857727668 6847162.8561709905),652652.985773,6.847163e+06,True,0.0


In [ ]:
addCellId2Trip(CellsDF,df)

In [ ]:
df

,agentID,trip_id,<route>_distance,start_link,start_x,start_y,end_link,end_x,end_y,vehicle_id,<leg>_trav_time,wait_time,<leg>_dep_time,origin,destination,geometry,tripType,Hub,originCell
272,10004937,10004937_1,4798,537171,640676.987241,6.845386e+06,255857,638650.536115,6.848612e+06,vehicle_drt_93,443,277,28822,POINT (640676.9872406526 6845385.660893883),POINT (638650.5361152911 6848611.9949821625),POINT (640676.987 6845385.661),egress,Hub_5,120
1215,1001879,1001879_1,2394,30347,645334.224928,6.847593e+06,249276,645178.745516,6.848194e+06,vehicle_drt_262,417,61,27593,POINT (645334.2249275153 6847592.524996355),POINT (645178.7455157 6848193.557723006),POINT (645334.225 6847592.525),egress,Hub_10,234
3190,10035536,10035536_2,780,373905,634006.318125,6.844592e+06,418142,633676.048264,6.844983e+06,vehicle_drt_139,136,161,53076,POINT (634006.3181254587 6844591.817313871),POINT (633676.0482636443 6844983.164553668),POINT (634006.318 6844591.817),access,Hub_0,4
3579,10038603,10038603_3,734,28293,642930.633995,6.849632e+06,523336,643382.681350,6.849181e+06,vehicle_drt_22,121,96,41275,POINT (642930.6339947096 6849631.82615489),POINT (643382.6813501306 6849180.759473929),POINT (642930.634 6849631.826),access,Hub_7,174
4260,10043934,10043934_1,1141,52986,644475.791845,6.846566e+06,492973,644505.725852,6.845947e+06,vehicle_drt_259,208,16,28345,POINT (644475.7918446767 6846565.807303847),POINT (644505.725852222 6845946.915065141),POINT (644475.792 6846565.807),egress,Hub_9,213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4173637,998386,998386_1,10051,363799,642117.061658,6.845355e+06,246639,642372.295495,6.843694e+06,vehicle_drt_167,1557,198,30118,POINT (642117.061658004 6845354.874643082),POINT (642372.2954946852 6843693.965287579),POINT (642117.062 6845354.875),egress,Hub_6,152
4174587,9998790,9998790_2,6887,255857,638650.536115,6.848612e+06,537171,640676.987241,6.845386e+06,vehicle_drt_125,682,178,56718,POINT (638650.5361152911 6848611.9949821625),POINT (640676.9872406526 6845385.660893883),POINT (638650.536 6848611.995),access,Hub_5,75
4174600,999887,999887_1,2818,13460,639578.460420,6.844363e+06,553637,641211.307923,6.842971e+06,vehicle_drt_39,483,551,30480,POINT (639578.4604200339 6844363.259593032),POINT (641211.3079227756 6842971.122922318),POINT (639578.460 6844363.260),egress,Hub_4,101
4174612,9998939,9998939_1,4798,537171,640676.987241,6.845386e+06,255857,638650.536115,6.848612e+06,vehicle_drt_101,384,457,31222,POINT (640676.9872406526 6845385.660893883),POINT (638650.5361152911 6848611.9949821625),POINT (640676.987 6845385.661),egress,Hub_5,120


In [ ]:
len(df["originCell"].unique())

217

In [ ]:
CellsDF.to_csv("/content/drive/MyDrive/diepolder's paper/library_pipeline/Paris/output/drt_cells_v1.csv",sep=';')

In [ ]:
df.to_csv("/content/drive/MyDrive/diepolder's paper/library_pipeline/Paris/output/drt_trips.csv",sep=';')

In [ ]:
import logging

import numpy as np
from pyinterpolate import read_txt, build_experimental_variogram, build_theoretical_variogram, kriging
import ExtractDrtTrips as edt
import pandas as pd
import SpacialOperations as sops
import logging
import tqdm

# length of interval in h E.g. 1 hour intervals --> interval = 1
# timeInterval gives the start and end of analysis. E.g. from 6am to 8pm  --> timeInterval = [06:00:00, 20:00:00]
def runAllIntervals(df,cellsDF,interval,timeInterval,hubsDf):
    logging.info('Starting interpolation of travel and wait times')
    cuts = edt.timeFrame2intervals(timeInterval,interval)
    progress = tqdm.tqdm(total=len(cuts)-1, desc='Interpolation for time interval', position=0)
    input = cellsDF.copy()
    for i in range(0,len(cuts)-1):
        progress.update(1)
        if i > 0:
            print()
            print('Interval ' + str(i+1))
            outDF = run(df, input, [cuts[i], cuts[i+1]], hubsDf)
            if len(outDF) == 0:
                pass
            else:
                outDF['Interval_start'] = [cuts[i]] * len(outDF)
                outDF['Interval_end'] = [cuts[i + 1]] * len(outDF)
                cellsDF = pd.concat([cellsDF, outDF], ignore_index=True)
        else:
            print()
            print('Interval 1')
            cellsDF = run(df, input, [cuts[i], cuts[i+1]], hubsDf)
            if len(cellsDF) == 0:
                cellsDF = input.copy()
                cellsDF.drop(df.index, inplace=True)
            else:
                cellsDF['Interval_start'] = [cuts[i]] * len(cellsDF)
                cellsDF['Interval_end'] = [cuts[i + 1]] * len(cellsDF)
    return cellsDF



def run(df,cellsDF,timeInterval,hubsDf):
    hubs = df['Hub'].unique()
    intervalStart = timeInterval[0]
    intervalEnd = timeInterval[1]
    print('Interpolation for Time interval ' + str(int(intervalStart/3600))+ ':' + str(int((intervalStart%3600)/60)) + ':00 to ' + str(int(intervalEnd/3600))+ ':' + str(int((intervalEnd%3600)/60))+':00')
    for hub in hubs:
        #Filter for current evaluated hub
        dfFiltered = df[(df['Hub'] == hub)]

        convexHullPoly = dfFiltered.unary_union.convex_hull.buffer(250)
        cellsFiltered = FilterCells2(convexHullPoly, cellsDF)

        # Filter for time interval
        df_ac_eg = dfFiltered[(dfFiltered['<leg>_dep_time'] > intervalStart) &
                        (dfFiltered['<leg>_dep_time'] < intervalEnd)][['start_x', 'start_y','end_x', 'end_y', '<leg>_trav_time', 'wait_time', 'tripType']]

        print("df_ac_eg",df_ac_eg)

        print(hub)
        # Interpolating Access Trips
        df_input = df_ac_eg[df_ac_eg['tripType'] == 'access']
        print('Interpolating Access Trips')
        print('Found ' + str(len(df_input)) + ' trips to interpolate')
        if len(df_input) < 2:
            print('Time interval has not enough entries for ' + hub)
            #continue
        else:
            typ = '<leg>_trav_time'
            dfUnique = df_input.groupby(by=['start_x', 'start_y'], as_index=False)[typ].mean()
            dfUnique.columns = ['start_x','start_y',typ]
            cellsDF = interpolate(hub,typ,dfUnique.to_numpy(),cellsDF,cellsFiltered,'ac')

            typ = 'wait_time'
            dfUnique = df_input.groupby(by=['start_x', 'start_y'], as_index=False)[typ].mean()
            dfUnique.columns = ['start_x','start_y',typ]
            cellsDF = interpolate(hub,typ,dfUnique.to_numpy(),cellsDF, cellsFiltered, 'ac')

        # Interpolating Egress Trips
        df_input = df_ac_eg[df_ac_eg['tripType'] == 'egress']
        print('Interpolating Egress Trips')
        print('Found ' + str(len(df_input)) + ' trips to interpolate')
        if len(df_input) < 2:
            print('Time interval has not enough entries for ' + hub)
            #continue
        else:
            typ = '<leg>_trav_time'
            dfUnique = df_input.groupby(by=['end_x', 'end_y'], as_index=False)[typ].mean()
            if len(dfUnique[typ].unique())<2:
                dfUnique.loc[0,typ] = dfUnique.loc[0,typ]+1
            dfUnique.columns = ['end_x', 'end_y', typ]
            cellsDF = interpolate(hub, typ, dfUnique.to_numpy(), cellsDF, cellsFiltered, 'eg')

            typ = 'wait_time'
            dfUnique = df_input.groupby(by=['end_x', 'end_y'], as_index=False)[typ].mean()
            if len(dfUnique[typ].unique())<2:
                dfUnique.loc[0,typ] = dfUnique.loc[0,typ]+1
            dfUnique.columns = ['end_x', 'end_y', typ]
            cellsDF = interpolate(hub, typ, dfUnique.to_numpy(), cellsDF, cellsFiltered, 'eg')


    return cellsDF

def interpolate(runID,typ,data,cellsDF, cellsFiltered, direction):
    unknown_point = cellsFiltered[['Centroid_X', 'Centroid_Y']].to_numpy()
    try:
        step_radius = 500  # meters
        max_range = 3000  # meters
        print('trying step: '+ str(step_radius))
        exp_semivar = build_experimental_variogram(input_array=data, step_size=step_radius, max_range=max_range)
    except:
        try:
            step_radius = 1500  # meters
            max_range = 3000  # meters
            print('trying step: ' + str(step_radius))
            exp_semivar = build_experimental_variogram(input_array=data, step_size=step_radius, max_range=max_range)
        except:
            step_radius = 3000  # meters
            max_range = 3000  # meters
            print('trying step: ' + str(step_radius))
            exp_semivar = build_experimental_variogram(input_array=data, step_size=step_radius, max_range=max_range)

    semivar = build_theoretical_variogram(experimental_variogram=exp_semivar,model_name="linear",
                                          sill=exp_semivar.variance, rang=3000)
    prediction = kriging(observations=data,
                         theoretical_model=semivar,
                         points=unknown_point,
                         how='ok',
                         no_neighbors=20)

    predictionsDF = pd.DataFrame(prediction,columns = [str(runID +'_'+typ+'_'+direction), str(runID +'_'+typ+'_'+ direction) + '_var_err', 'x','y'])
    df = cellsDF.merge(predictionsDF,left_on=['Centroid_X', 'Centroid_Y'], right_on=['x','y'], how= 'left')
    df.fillna('-999', inplace=True)
    df.drop(columns=['x','y'],inplace=True)
    return df

def cellDist2Hub(cellsDf, radius, hubLocation):
    cellX = cellsDf['Centroid_X']
    cellY = cellsDf['Centroid_Y']
    cellDist = []
    for i in range(len(cellX)):
        cellDist.append(sops.pointinCircle(radius,hubLocation,[cellX[i],cellY[i]]))
    return cellDist

def FilterCells(cellsDf, radius, hubLocation):
    cellsDf['relevant'] = cellDist2Hub(cellsDf, radius, hubLocation)
    cellsFiltered = cellsDf[cellsDf['relevant']==True]
    cellsDf.drop(columns=['relevant'], inplace=True)
    return cellsFiltered

def FilterCells2(convexHullPoly, cellsDF):
    cdf_f = cellsDF.copy()
    cdf_f['TripCatchement'] = cdf_f['geometry'].overlaps(convexHullPoly) | cdf_f['geometry'].within(convexHullPoly)
    cdf_f = cdf_f[cdf_f['TripCatchement'] == True]
    cdf_f.reset_index(inplace = True)
    return cdf_f

def getHubLocation(HubsDf,hubName):
    locationWKT = HubsDf[HubsDf['name']==hubName]['position'].values.tolist()
    coord = locationWKT[0].split(' ')
    del coord[0]
    coord[0] = coord[0].lstrip('(')
    coord[1] = coord[1].rstrip(')')
    return [float(coord[0]),float(coord[1])]






In [ ]:
l=[start_time, end_time]
l

['07:00:00', '19:00:00']

In [ ]:
cuts = edt.timeFrame2intervals(l,deltaT)
cuts


In [ ]:
outDF = run(df, input, [cuts[0], cuts[0+1]], DfHubs)

In [ ]:
hubs = df['Hub'].unique()
intervalStart = cuts[0]
intervalEnd = cuts[1]
hubs

array(['Hub_5', 'Hub_10', 'Hub_0', 'Hub_7', 'Hub_9', 'Hub_4', 'Hub_6',
       'Hub_8', 'Hub_13', 'Hub_3', 'Hub_12', 'Hub_11', 'Hub_1', 'Hub_14',
       'Hub_15', 'Hub_2'], dtype=object)

In [ ]:
dfFiltered = df[(df['Hub'] == hubs[0])]
dfFiltered


,agentID,trip_id,<route>_distance,start_link,start_x,start_y,end_link,end_x,end_y,vehicle_id,<leg>_trav_time,wait_time,<leg>_dep_time,origin,destination,geometry,tripType,Hub,originCell
272,10004937,10004937_1,4798,537171,640676.987241,6.845386e+06,255857,638650.536115,6.848612e+06,vehicle_drt_93,443,277,28822,POINT (640676.9872406526 6845385.660893883),POINT (638650.5361152911 6848611.9949821625),POINT (640676.987 6845385.661),egress,Hub_5,120
7287,10068547,10068547_3,1999,97873,640837.983781,6.844600e+06,537171,640676.987241,6.845386e+06,vehicle_drt_95,340,344,58529,POINT (640837.9837812549 6844600.024697466),POINT (640676.9872406526 6845385.660893883),POINT (640837.984 6844600.025),access,Hub_5,119
18552,1018866,1018866_1,2576,537171,640676.987241,6.845386e+06,16183,640448.222324,6.844690e+06,vehicle_drt_103,437,121,23782,POINT (640676.9872406526 6845385.660893883),POINT (640448.2223237279 6844689.853327539),POINT (640676.987 6845385.661),egress,Hub_5,120
24568,10231440,10231440_2,7356,255857,638650.536115,6.848612e+06,537171,640676.987241,6.845386e+06,vehicle_drt_104,664,94,59230,POINT (638650.5361152911 6848611.9949821625),POINT (640676.9872406526 6845385.660893883),POINT (638650.536 6848611.995),access,Hub_5,75
26023,1024408,1024408_2,5585,14899,639238.957909,6.848200e+06,537171,640676.987241,6.845386e+06,vehicle_drt_107,656,149,67076,POINT (639238.9579085483 6848200.140838478),POINT (640676.9872406526 6845385.660893883),POINT (639238.958 6848200.141),access,Hub_5,89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171725,9958522,9958522_1,4496,537171,640676.987241,6.845386e+06,97873,640837.983781,6.844600e+06,vehicle_drt_24,818,138,42022,POINT (640676.9872406526 6845385.660893883),POINT (640837.9837812549 6844600.024697466),POINT (640676.987 6845385.661),egress,Hub_5,120
4171926,9961598,9961598_1,3633,537171,640676.987241,6.845386e+06,97873,640837.983781,6.844600e+06,vehicle_drt_176,654,153,32302,POINT (640676.9872406526 6845385.660893883),POINT (640837.9837812549 6844600.024697466),POINT (640676.987 6845385.661),egress,Hub_5,120
4171929,9961598,9961598_2,1999,97873,640837.983781,6.844600e+06,537171,640676.987241,6.845386e+06,vehicle_drt_95,340,277,58596,POINT (640837.9837812549 6844600.024697466),POINT (640676.9872406526 6845385.660893883),POINT (640837.984 6844600.025),access,Hub_5,119
4174587,9998790,9998790_2,6887,255857,638650.536115,6.848612e+06,537171,640676.987241,6.845386e+06,vehicle_drt_125,682,178,56718,POINT (638650.5361152911 6848611.9949821625),POINT (640676.9872406526 6845385.660893883),POINT (638650.536 6848611.995),access,Hub_5,75


In [ ]:
convexHullPoly = dfFiltered.unary_union.convex_hull.buffer(250)

In [ ]:
convexHullPoly = dfFiltered.unary_union.convex_hull.buffer(250)

cellsFiltered = FilterCells2(convexHullPoly, CellsDF.copy())
cellsFiltered

,index,cell_id,geometry,Centroids,Centroid_X,Centroid_Y,inStudyArea,origin_count,TripCatchement
0,72,72,"POLYGON ((638934.000 6845992.000, 638689.000 6...",POINT (638435.2844789247 6845997.315878143),638435.284479,6.845997e+06,True,18.0,True
1,73,73,"POLYGON ((638943.000 6846858.000, 638698.000 6...",POINT (638444.4593163949 6846863.108410401),638444.459316,6.846863e+06,True,0.0,True
2,74,74,"POLYGON ((638952.000 6847724.000, 638707.000 6...",POINT (638453.6342150941 6847728.9067206),638453.634215,6.847729e+06,True,0.0,True
3,75,75,"POLYGON ((638961.000 6848589.000, 638717.000 6...",POINT (638462.8091751989 6848594.710825397),638462.809175,6.848595e+06,True,31.0,True
4,76,76,"POLYGON ((638970.000 6849455.000, 638726.000 6...",POINT (638471.9841968861 6849460.520741464),638471.984197,6.849461e+06,True,3.0,True
5,77,77,"POLYGON ((638979.000 6850321.000, 638735.000 6...",POINT (638481.159280332 6850326.336485467),638481.159280,6.850326e+06,True,0.0,True
6,86,86,"POLYGON ((639677.000 6845551.000, 639432.000 6...",POINT (639178.4872532784 6845556.545539746),639178.487253,6.845557e+06,True,19.0,True
7,87,87,"POLYGON ((639686.000 6846417.000, 639441.000 6...",POINT (639187.5506275527 6846422.33636297),639187.550628,6.846422e+06,True,1.0,True
8,88,88,"POLYGON ((639695.000 6847283.000, 639450.000 6...",POINT (639196.6140622252 6847288.132955808),639196.614062,6.847288e+06,True,1.0,True
9,89,89,"POLYGON ((639704.000 6848149.000, 639459.000 6...",POINT (639205.6775574704 6848153.935334923),639205.677557,6.848154e+06,True,16.0,True


In [ ]:
df_ac_eg = dfFiltered[(dfFiltered['<leg>_dep_time'] > intervalStart) &
                        (dfFiltered['<leg>_dep_time'] < intervalEnd)][['start_x', 'start_y','end_x', 'end_y', '<leg>_trav_time', 'wait_time', 'tripType']]

df_input = df_ac_eg[df_ac_eg['tripType'] == 'access']
df_input
# if len(df_input) < 2:dfUnique
#   print("trip exists")

,start_x,start_y,end_x,end_y,<leg>_trav_time,wait_time,tripType
129660,640164.096445,6.844959e+06,640676.987241,6.845386e+06,344,147,access
2325164,639075.105613,6.848155e+06,640676.987241,6.845386e+06,585,211,access
2329530,639018.377704,6.848157e+06,640676.987241,6.845386e+06,831,177,access
2401990,640722.778403,6.845505e+06,640676.987241,6.845386e+06,71,69,access
2409076,641071.288710,6.844412e+06,640676.987241,6.845386e+06,512,77,access
2413885,640108.202876,6.844900e+06,640676.987241,6.845386e+06,372,281,access
2415796,640238.886818,6.845619e+06,640676.987241,6.845386e+06,251,334,access
2416755,640048.480086,6.845650e+06,640676.987241,6.845386e+06,268,284,access
2434723,640507.333138,6.845225e+06,640676.987241,6.845386e+06,219,433,access
2436341,640802.659130,6.845489e+06,640676.987241,6.845386e+06,79,110,access


In [ ]:
len(df_input["start_x"].values)

40

In [ ]:
typ = '<leg>_trav_time'
dfUnique = df_input.groupby(by=['end_x', 'end_y'], as_index=False)[typ].mean()
dfUnique
# if len(dfUnique[typ].unique())<2:
#   print("true")
#   dfUnique.loc[0,typ] = dfUnique.loc[0,typ]+1
# dfUnique
# dfUnique.columns = ['end_x', 'end_y', typ]
# cellsDF = interpolate(hub, typ, dfUnique.to_numpy(), cellsDF, cellsFiltered, 'eg')

,end_x,end_y,<leg>_trav_time
0,640676.987241,6.845386e+06,333.925


In [ ]:
if len(dfUnique[typ].unique())<2:
  print("true")
  dfUnique.loc[0,typ] = dfUnique.loc[0,typ]+1
dfUnique

true


,end_x,end_y,<leg>_trav_time
0,640676.987241,6.845386e+06,334.925


In [ ]:
dfUnique.to_numpy()

array([[6.40676987e+05, 6.84538566e+06, 3.34925000e+02]])

In [ ]:
CellsDF

,cell_id,geometry,Centroids,Centroid_X,Centroid_Y,inStudyArea,origin_count
0,0,"POLYGON ((633685.000 6844751.000, 633440.000 6...",POINT (633186.2403781626 6844756.47498971),633186.240378,6.844756e+06,True,2.0
1,1,"POLYGON ((634399.000 6841712.000, 634154.000 6...",POINT (633899.683422456 6841717.750894942),633899.683422,6.841718e+06,True,0.0
2,2,"POLYGON ((634408.000 6842578.000, 634164.000 6...",POINT (633909.5265291672 6842583.507444308),633909.526529,6.842584e+06,True,1.0
3,3,"POLYGON ((634418.000 6843444.000, 634174.000 6...",POINT (633919.3697006224 6843449.269688284),633919.369701,6.843449e+06,True,1.0
4,4,"POLYGON ((634428.000 6844309.000, 634183.000 6...",POINT (633929.2129370112 6844315.037643521),633929.212937,6.844315e+06,True,55.0
...,...,...,...,...,...,...,...
351,351,"POLYGON ((652407.000 6847598.000, 652162.000 6...",POINT (651908.9867740034 6847601.907483266),651908.986774,6.847602e+06,True,0.0
352,352,"POLYGON ((652414.000 6848464.000, 652169.000 6...",POINT (651916.1558914991 6848467.73052034),651916.155891,6.848468e+06,True,0.0
353,353,"POLYGON ((652422.000 6849329.000, 652176.000 6...",POINT (651923.3250571139 6849333.5593688125),651923.325057,6.849334e+06,True,0.0
354,354,"POLYGON ((653151.000 6847159.000, 652906.000 6...",POINT (652652.9857727668 6847162.8561709905),652652.985773,6.847163e+06,True,0.0


In [ ]:
cellsFiltered

,index,cell_id,geometry,Centroids,Centroid_X,Centroid_Y,inStudyArea,origin_count,TripCatchement
0,72,72,"POLYGON ((638934.000 6845992.000, 638689.000 6...",POINT (638435.2844789247 6845997.315878143),638435.284479,6.845997e+06,True,18.0,True
1,73,73,"POLYGON ((638943.000 6846858.000, 638698.000 6...",POINT (638444.4593163949 6846863.108410401),638444.459316,6.846863e+06,True,0.0,True
2,74,74,"POLYGON ((638952.000 6847724.000, 638707.000 6...",POINT (638453.6342150941 6847728.9067206),638453.634215,6.847729e+06,True,0.0,True
3,75,75,"POLYGON ((638961.000 6848589.000, 638717.000 6...",POINT (638462.8091751989 6848594.710825397),638462.809175,6.848595e+06,True,31.0,True
4,76,76,"POLYGON ((638970.000 6849455.000, 638726.000 6...",POINT (638471.9841968861 6849460.520741464),638471.984197,6.849461e+06,True,3.0,True
5,77,77,"POLYGON ((638979.000 6850321.000, 638735.000 6...",POINT (638481.159280332 6850326.336485467),638481.159280,6.850326e+06,True,0.0,True
6,86,86,"POLYGON ((639677.000 6845551.000, 639432.000 6...",POINT (639178.4872532784 6845556.545539746),639178.487253,6.845557e+06,True,19.0,True
7,87,87,"POLYGON ((639686.000 6846417.000, 639441.000 6...",POINT (639187.5506275527 6846422.33636297),639187.550628,6.846422e+06,True,1.0,True
8,88,88,"POLYGON ((639695.000 6847283.000, 639450.000 6...",POINT (639196.6140622252 6847288.132955808),639196.614062,6.847288e+06,True,1.0,True
9,89,89,"POLYGON ((639704.000 6848149.000, 639459.000 6...",POINT (639205.6775574704 6848153.935334923),639205.677557,6.848154e+06,True,16.0,True


In [ ]:
dfUnique.columns = ['end_x', 'end_y', typ]

cellsDF2 = interpolate(hubs[0], typ, dfUnique.to_numpy(),CellsDF.copy() , cellsFiltered, 'ac')


/usr/local/lib/python3.10/dist-packages/pyinterpolate/variogram/utils/metrics.py:188: UndefinedSMAPEWarning: 'Values of observation and prediction on position 0 are equal to zero. Partial smape is set to 0 to avoid zero by zero division.'
  warnings.warn(msg, UndefinedSMAPEWarning)


trying step: 500
trying step: 1500



100%|██████████| 27/27 [00:00<00:00, 6434.81it/s]

    Hub_5_<leg>_trav_time_ac  Hub_5_<leg>_trav_time_ac_var_err              x  \
0                    334.925                               0.0  638435.284479   
1                    334.925                               0.0  638444.459316   
2                    334.925                               0.0  638453.634215   
3                    334.925                               0.0  638462.809175   
4                    334.925                               0.0  638471.984197   
5                    334.925                               0.0  638481.159280   
6                    334.925                               0.0  639178.487253   
7                    334.925                               0.0  639187.550628   
8                    334.925                               0.0  639196.614062   
9                    334.925                               0.0  639205.677557   
10                   334.925                               0.0  639214.741113   
11                   334.925

In [ ]:
cellsDF2

,cell_id,geometry,Centroids,Centroid_X,Centroid_Y,inStudyArea,origin_count,Hub_5_<leg>_trav_time_ac,Hub_5_<leg>_trav_time_ac_var_err
0,0,"POLYGON ((633685.000 6844751.000, 633440.000 6...",POINT (633186.2403781626 6844756.47498971),633186.240378,6.844756e+06,True,2.0,-999,-999
1,1,"POLYGON ((634399.000 6841712.000, 634154.000 6...",POINT (633899.683422456 6841717.750894942),633899.683422,6.841718e+06,True,0.0,-999,-999
2,2,"POLYGON ((634408.000 6842578.000, 634164.000 6...",POINT (633909.5265291672 6842583.507444308),633909.526529,6.842584e+06,True,1.0,-999,-999
3,3,"POLYGON ((634418.000 6843444.000, 634174.000 6...",POINT (633919.3697006224 6843449.269688284),633919.369701,6.843449e+06,True,1.0,-999,-999
4,4,"POLYGON ((634428.000 6844309.000, 634183.000 6...",POINT (633929.2129370112 6844315.037643521),633929.212937,6.844315e+06,True,55.0,-999,-999
...,...,...,...,...,...,...,...,...,...
351,351,"POLYGON ((652407.000 6847598.000, 652162.000 6...",POINT (651908.9867740034 6847601.907483266),651908.986774,6.847602e+06,True,0.0,-999,-999
352,352,"POLYGON ((652414.000 6848464.000, 652169.000 6...",POINT (651916.1558914991 6848467.73052034),651916.155891,6.848468e+06,True,0.0,-999,-999
353,353,"POLYGON ((652422.000 6849329.000, 652176.000 6...",POINT (651923.3250571139 6849333.5593688125),651923.325057,6.849334e+06,True,0.0,-999,-999
354,354,"POLYGON ((653151.000 6847159.000, 652906.000 6...",POINT (652652.9857727668 6847162.8561709905),652652.985773,6.847163e+06,True,0.0,-999,-999


In [ ]:
cellsDF2["Hub_5_<leg>_trav_time_ac"].values

array(['-999', '-999', '-999', '-999', '-999', '-999', '-999', '-999',
       '-999', '-999', '-999', '-999', '-999', '-999', '-999', '-999',
       '-999', '-999', '-999', '-999', '-999', '-999', '-999', '-999',
       '-999', '-999', '-999', '-999', '-999', '-999', '-999', '-999',
       '-999', '-999', '-999', '-999', '-999', '-999', '-999', '-999',
       '-999', '-999', '-999', '-999', '-999', '-999', '-999', '-999',
       '-999', '-999', '-999', '-999', '-999', '-999', '-999', '-999',
       '-999', '-999', '-999', '-999', '-999', '-999', '-999', '-999',
       '-999', '-999', '-999', '-999', '-999', '-999', '-999', '-999',
       334.925, 334.925, 334.925, 334.925, 334.925, 334.925, '-999',
       '-999', '-999', '-999', '-999', '-999', '-999', '-999', 334.925,
       334.925, 334.925, 334.925, 334.925, 334.925, '-999', '-999',
       '-999', '-999', '-999', '-999', '-999', '-999', '-999', 334.925,
       334.925, 334.925, 334.925, 334.925, 334.925, '-999', '-999',
       '-999

In [ ]:
unknown_point = cellsFiltered[['Centroid_X', 'Centroid_Y']].to_numpy()
len(unknown_point)

27

27

In [ ]:
def interpolate(runID,typ,data,cellsDF, cellsFiltered, direction):
    unknown_point = cellsFiltered[['Centroid_X', 'Centroid_Y']].to_numpy()
    try:
        step_radius = 500  # meters
        max_range = 3000  # meters
        print('trying step: '+ str(step_radius))
        exp_semivar = build_experimental_variogram(input_array=data, step_size=step_radius, max_range=max_range)
    except:
        try:
            step_radius = 1500  # meters
            max_range = 3000  # meters
            print('trying step: ' + str(step_radius))
            exp_semivar = build_experimental_variogram(input_array=data, step_size=step_radius, max_range=max_range)
        except:
            step_radius = 3000  # meters
            max_range = 3000  # meters
            print('trying step: ' + str(step_radius))
            exp_semivar = build_experimental_variogram(input_array=data, step_size=step_radius, max_range=max_range)

    semivar = build_theoretical_variogram(experimental_variogram=exp_semivar,model_name="linear",
                                          sill=exp_semivar.variance, rang=3000)
    prediction = kriging(observations=data,
                         theoretical_model=semivar,
                         points=unknown_point,
                         how='ok',
                         no_neighbors=20)

    predictionsDF = pd.DataFrame(prediction,columns = [str(runID +'_'+typ+'_'+direction), str(runID +'_'+typ+'_'+ direction) + '_var_err', 'x','y'])
    print(predictionsDF)
    # df = cellsDF.merge(predictionsDF,left_on=['Centroid_X', 'Centroid_Y'], right_on=['x','y'], how= 'left')
    # df.fillna('-999', inplace=True)
    # df.drop(columns=['x','y'],inplace=True)
    return predictionsDF #df


In [ ]:
df_input

,start_x,start_y,end_x,end_y,<leg>_trav_time,wait_time,tripType
129660,640164.096445,6.844959e+06,640676.987241,6.845386e+06,344,147,access
2325164,639075.105613,6.848155e+06,640676.987241,6.845386e+06,585,211,access
2329530,639018.377704,6.848157e+06,640676.987241,6.845386e+06,831,177,access
2401990,640722.778403,6.845505e+06,640676.987241,6.845386e+06,71,69,access
2409076,641071.288710,6.844412e+06,640676.987241,6.845386e+06,512,77,access
2413885,640108.202876,6.844900e+06,640676.987241,6.845386e+06,372,281,access
2415796,640238.886818,6.845619e+06,640676.987241,6.845386e+06,251,334,access
2416755,640048.480086,6.845650e+06,640676.987241,6.845386e+06,268,284,access
2434723,640507.333138,6.845225e+06,640676.987241,6.845386e+06,219,433,access
2436341,640802.659130,6.845489e+06,640676.987241,6.845386e+06,79,110,access


In [ ]:
typ = '<leg>_trav_time'
dfUnique = df_input.groupby(by=['end_x', 'end_y'], as_index=False)[typ].mean()
dfUnique

,end_x,end_y,<leg>_trav_time
0,640676.987241,6.845386e+06,333.925


In [ ]:
typ = '<leg>_trav_time'
cellsFiltered=df_input['end_x', 'end_y', typ]
# dfUnique.columns = ['end_x', 'end_y', typ]

res = interpolate(hubs[0], typ, dfUnique.to_numpy(),CellsDF.copy() , cellsFiltered, 'ac')

In [ ]:
CellsDF = runAllIntervals(df, CellsDF, deltaT, [start_time, end_time], DfHubs)

Interpolation for time interval:   0%|          | 0/12 [00:00<?, ?it/s]


Interval 1
Interpolation for Time interval 7:0:00 to 8:0:00
df_ac_eg                start_x       start_y          end_x         end_y  \
129660   640164.096445  6.844959e+06  640676.987241  6.845386e+06   
139991   640676.987241  6.845386e+06  640939.627102  6.845178e+06   
141841   640676.987241  6.845386e+06  640939.627102  6.845178e+06   
142720   640676.987241  6.845386e+06  640939.627102  6.845178e+06   
175319   640676.987241  6.845386e+06  640448.222324  6.844690e+06   
...                ...           ...            ...           ...   
4061616  640676.987241  6.845386e+06  640837.983781  6.844600e+06   
4083296  640676.987241  6.845386e+06  638650.536115  6.848612e+06   
4083429  640676.987241  6.845386e+06  640592.929526  6.845597e+06   
4096871  640676.987241  6.845386e+06  638650.536115  6.848612e+06   
4121723  640676.987241  6.845386e+06  638650.536115  6.848612e+06   

         <leg>_trav_time  wait_time tripType  
129660               344        147   access  
139991 


100%|██████████| 27/27 [00:00<00:00, 1787.97it/s]


trying step: 500



100%|██████████| 27/27 [00:00<00:00, 1873.94it/s]


...

trying step: 500
trying step: 1500



100%|██████████| 38/38 [00:00<00:00, 2936.27it/s]


Interpolating Egress Trips
Found 14 trips to interpolate
trying step: 500



100%|██████████| 38/38 [00:00<00:00, 3401.71it/s]


KeyboardInterrupt: 

In [ ]:
drtGTFS = createGTFS.run(CellsDF, DfHubs, deltaT, [start_time, end_time])

adding all stops to gtfs: stops.txt: 100%|██████████| 469/469 [00:00<00:00, 1563.11it/s]
adding all waiting and travel times to gtfs: stop_times.txt:   0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as RGF93 v1 / Lambert-93 (the single non-null crs provided).
  warnings.warn(
adding all waiting and travel times to gtfs: stop_times.txt:  34%|███▍      | 160/469 [00:04<00:09, 34.13it/s]

KeyboardInterrupt: 